### Swipe(엄지와 검지를 붙여서 오른쪽으로 스와이프 / 음악 추천 기능 호출) 제스처 학습을 위한 데이터 복제

In [6]:
# 랜드마크 CSV 기반 시퀀스 데이터 확장 코드
import pandas as pd
import numpy as np

In [7]:
# 🔹 설정
original_csv_path = '/home/jieun/Gesture_Recognition/test/data_gesture_Swipe.csv'  # 원본 CSV 경로
output_csv_path = '/home/jieun/Gesture_Recognition/test/data_gesture_1_Swipe_extended.csv'  # 저장할 CSV 경로
sequence_length = 30     # 시퀀스 길이
num_sequences = 6000     # 생성할 시퀀스 수
new_label = 7            # 추가될 제스처 라벨 (None 제외하고 기존 0~6 이었으므로 7로 지정)

In [8]:
# 🔹 CSV 로드
df = pd.read_csv(original_csv_path)

# 🔹 특징과 라벨 분리
features = df.iloc[:, :-1].values  # shape: (N, 99)
label_column = np.full((sequence_length, 1), new_label)

# 🔹 시퀀스 생성
sequence_list = []

np.random.seed(42)  # 재현 가능성을 위해
for _ in range(num_sequences):
    indices = np.random.choice(len(features), sequence_length, replace=True)
    sequence = features[indices]
    sequence_with_label = np.hstack((sequence, label_column))  # shape: (30, 100)
    sequence_list.append(sequence_with_label)
    
# 🔹 전체 합치기
final_data = np.vstack(sequence_list)  # shape: (6000 * 30, 100)
final_df = pd.DataFrame(final_data)

In [9]:
# 🔹 저장
final_df.to_csv(output_csv_path, index=False)
print(f"✅ {output_csv_path} 파일로 저장 완료. shape: {final_data.shape}")

✅ /home/jieun/Gesture_Recognition/test/data_gesture_1_Swipe_extended.csv 파일로 저장 완료. shape: (180000, 100)


### All_Stop 10개의 csv 파일을 각 17개씩 복제하여 총 177개의 csv 파일로 증강하는 코드

In [2]:
import os
import glob
import shutil

# 원본 CSV 파일들이 있는 경로
source_dir = '/home/jieun/Gesture_Recognition/test'
# 복제본을 저장할 경로
target_dir = '/home/jieun/Gesture_Recognition/augmented'
os.makedirs(target_dir, exist_ok=True)

# 원본 파일 리스트 (정렬 필요 시 정렬)
source_files = sorted(glob.glob(os.path.join(source_dir, 'data_gesture_*_All_Stop.csv')))

# 총 만들고자 하는 파일 수
target_total = 177
source_count = len(source_files)

# 몇 개씩 복제하면 되는지 계산
copies_per_file = target_total // source_count
remainder = target_total % source_count

print(f"원본 파일 수: {source_count}")
print(f"파일당 복제 수: {copies_per_file} (+1 추가할 파일 수: {remainder})")

file_index = 1

for idx, src_file in enumerate(source_files):
    # 이 파일은 몇 번 복제할지 결정
    num_copies = copies_per_file + (1 if idx < remainder else 0)

    for _ in range(num_copies):
        filename = f'data_gesture_All_Stop_{file_index:03d}.csv'
        target_path = os.path.join(target_dir, filename)
        shutil.copy(src_file, target_path)
        print(f'복제: {src_file} → {target_path}')
        file_index += 1

print(f"\n✅ 총 {file_index - 1}개 파일 생성 완료.")


원본 파일 수: 10
파일당 복제 수: 17 (+1 추가할 파일 수: 7)
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_001.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_002.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_003.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_004.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_005.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Recognition/augmented/data_gesture_All_Stop_006.csv
복제: /home/jieun/Gesture_Recognition/test/data_gesture_10_All_Stop.csv → /home/jieun/Gesture_Re

### All_Stop의 기존 35개 파일을 다양한 방법으로 augmentation 하여 총 5400개의 파일을 생성하는 코드

In [4]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

# 기본 설정
source_dir = '/home/jieun/Gesture_Recognition/test'      # 원본 35개 csv가 있는 폴더
target_dir = '/home/jieun/Gesture_Recognition/augmented_csv'   # 결과 저장 폴더
os.makedirs(target_dir, exist_ok=True)

# 원하는 총 증강 수
target_total = 5400

# 증강 조합 방식
def augment_shift_noise(data):
    """프레임 시프트 + 랜드마크 노이즈"""
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    noise = np.random.normal(0, 0.02, size=data[:, :99].shape)
    data[:, :99] += noise
    return data

def augment_shift_scale(data):
    """프레임 시프트 + 스케일 조정"""
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    scale = np.random.uniform(0.95, 1.05)
    data[:, :99] *= scale
    return data

def augment_repeat_insert(data):
    """랜덤 프레임 중복 삽입"""
    insert_idx = random.randint(0, len(data)-1)
    frame_to_insert = data[insert_idx:insert_idx+1]
    insert_pos = random.randint(0, len(data))
    data = np.insert(data, insert_pos, frame_to_insert, axis=0)
    if len(data) > 30:
        data = data[:30]
    return data

# 원본 파일 로딩
file_list = sorted([f for f in os.listdir(source_dir) if f.endswith('.csv')])
source_count = len(file_list)

# 파일당 생성 수 계산
copies_per_file = target_total // source_count
remainder = target_total % source_count

print(f"원본 파일 수: {source_count}")
print(f"파일당 복제 수: {copies_per_file} (+1 파일: {remainder})")

file_index = 1

for i, fname in tqdm(enumerate(file_list), total=len(file_list)):
    fpath = os.path.join(source_dir, fname)
    base = pd.read_csv(fpath, header=None).to_numpy()
    
    # 항상 포함: 원본
    save_path = os.path.join(target_dir, f'data_gesture_{file_index:04d}_All_Stop.csv')
    np.savetxt(save_path, base, delimiter=',')
    file_index += 1

    # 복제 수 계산
    n = copies_per_file + (1 if i < remainder else 0)
    # n-1개 증강본 생성
    for _ in range(n - 1):
        aug_type = random.choice(['noise', 'scale', 'repeat'])

        if aug_type == 'noise':
            aug = augment_shift_noise(base.copy())
        elif aug_type == 'scale':
            aug = augment_shift_scale(base.copy())
        elif aug_type == 'repeat':
            aug = augment_repeat_insert(base.copy())

        save_path = os.path.join(target_dir, f'data_gesture_{file_index:04d}_All_Stop.csv')
        np.savetxt(save_path, aug, delimiter=',')
        file_index += 1

print(f"\n✅ 최종 생성된 파일 수: {file_index - 1}개")

원본 파일 수: 35
파일당 복제 수: 154 (+1 파일: 10)


  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 35/35 [00:07<00:00,  4.90it/s]


✅ 최종 생성된 파일 수: 5400개


### indj 제스처 데이터 증강 (각 파일마다 원본 포함 10개씩 랜덤한 방법으로 증강)

In [8]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

# 기본 설정
source_dir = '/home/jieun/Gesture_Recognition/test_data_2'      # 원본 143개 csv가 있는 폴더
target_dir = '/home/jieun/Gesture_Recognition/augmented_csv_indj_test_2'   # 결과 저장 폴더
os.makedirs(target_dir, exist_ok=True)

# 증강 함수
'''프레임 시프트 + 관절 노이즈 추가'''
def augment_shift_noise(data):
    shift = random.randint(-2, 2)
    # 프레임 순서를 앞뒤로 +-2 이내로 이동
    data = np.roll(data, shift, axis=0)
    # 관절값(x, y, z)에 평균 0, 표준편차 0.02의 노이즈 추가
    noise = np.random.normal(0, 0.02, size=data[:, :99].shape)
    data[:, :99] += noise
    return data

'''프레임 시프트 + 스케일 조정'''
def augment_shift_scale(data):
    # 프레임 순서를 앞뒤로 +-2 이내로 이동
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    # 랜드마크 값에 0.95~1.05 사이의 랜덤 스케일 적용
    scale = np.random.uniform(0.95, 1.05)
    data[:, :99] *= scale
    return data

'''랜덤 프레임 복제 삽입'''
def augment_repeat_insert(data):
    # 임의의 프레임 1개 복제
    insert_idx = random.randint(0, len(data)-1)
    # 랜덤 위치에 삽입
    frame_to_insert = data[insert_idx:insert_idx+1]
    insert_pos = random.randint(0, len(data))
    data = np.insert(data, insert_pos, frame_to_insert, axis=0)
    # 만약 프레임이 30개를 초과하면 잘라냄
    if len(data) > 30:
        data = data[:30]
    return data

# 증강 시작
file_list = sorted([f for f in os.listdir(source_dir) if f.endswith('.csv')])
file_index = 1

for fname in tqdm(file_list, desc="증강 중"):
    fpath = os.path.join(source_dir, fname)
    base = pd.read_csv(fpath, header=None).to_numpy()

    # 원본 저장
    save_path = os.path.join(target_dir, f'data_gesture_{file_index:04d}_aug.csv')
    np.savetxt(save_path, base, delimiter=',')
    file_index += 1

    # 증강 9개 생성
    for _ in range(11):
        aug_type = random.choice(['noise', 'scale', 'repeat'])
        if aug_type == 'noise':
            aug = augment_shift_noise(base.copy())
        elif aug_type == 'scale':
            aug = augment_shift_scale(base.copy())
        else:
            aug = augment_repeat_insert(base.copy())

        save_path = os.path.join(target_dir, f'data_gesture_{file_index:04d}_aug.csv')
        np.savetxt(save_path, aug, delimiter=',')
        file_index += 1

print(f"\n✅ 총 생성된 파일 수: {file_index - 1}개 (각 원본당 10개)")

증강 중: 100%|██████████| 121/121 [00:02<00:00, 55.81it/s]


✅ 총 생성된 파일 수: 1452개 (각 원본당 10개)


### 한 파일 당 X10로 복제

In [5]:
import os
import shutil

source_dir = '/home/jieun/Gesture_Recognition/test_data'
target_dir = '/home/jieun/Gesture_Recognition/test_data_replicated'
os.makedirs(target_dir, exist_ok=True)

file_list = sorted([f for f in os.listdir(source_dir) if f.endswith('.csv')])

copies_per_file = 10
# file_index = 1

for fname in file_list:
    src_path = os.path.join(source_dir, fname)
    parts = fname.replace('.csv', '').split('_')
    if len(parts) < 4:
        print(f"⚠️ 파일명 포맷 이상: {fname}")
        continue
    gesture = parts[2]
    user_id = parts[3]
    
    for i in range(copies_per_file):
        new_fname = f'data_gesture_{gesture}_{user_id}_aug{i+1}.csv'
        dst_path = os.path.join(target_dir, new_fname)
        shutil.copy(src_path, dst_path)

print(f"✅ 총 생성 파일 수: {file_index - 1}개")


✅ 총 생성 파일 수: 1430개


### 프레임 수 정규화 : 30프레임만 유지

In [9]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 경로 설정
data_dir = '/home/jieun/Gesture_Recognition/augmented_csv_indj_test_2'  # 증강된 CSV 파일들이 있는 폴더
output_dir = '/home/jieun/Gesture_Recognition/augmented_csv_indj_test_3'
os.makedirs(output_dir, exist_ok=True)

file_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

for fname in tqdm(file_list):
    path = os.path.join(data_dir, fname)
    data = pd.read_csv(path, header=None).to_numpy()

    if data.shape[0] > 30:
        data = data[:30]  # 앞에서 30프레임만 사용

    elif data.shape[0] < 30:
        # 부족하면 마지막 프레임을 반복해서 채움
        pad = np.tile(data[-1:], (30 - data.shape[0], 1))
        data = np.vstack([data, pad])

    out_path = os.path.join(output_dir, fname)
    np.savetxt(out_path, data, delimiter=',')


100%|██████████| 1452/1452 [00:04<00:00, 342.74it/s]


### All_Stop 제스처 데이터를 30 프레임 -> 15프레임 등간격 추출

In [11]:
import os

source_dir = '../augmented_csv_indj_test_2/'
target_dir = '../augmented_csv_indj_test_2_frame_15/'
os.makedirs(target_dir, exist_ok=True)

# All_Stop 파일만 필터링
# file_list = sorted([
#     f for f in os.listdir(source_dir)
#     if 'All_Stop' in f and f.endswith('.csv')
# ])

# 모든 CSV 파일 필터링
file_list = sorted([
    f for f in os.listdir(source_dir)
    if f.endswith('.csv')
])

print(f"총 변환 대상 파일 수: {len(file_list)}개")

for i, fname in enumerate(file_list):
    src_path = os.path.join(source_dir, fname)
    dst_path = os.path.join(target_dir, fname)

    with open(src_path, 'r') as f:
        lines = f.readlines()

    if len(lines) != 30:
        print(f"⚠️ {fname}: 예상과 다른 프레임 수 ({len(lines)}) → 건너뜀")
        continue

    # ✅ 홀수 행만 선택 (0-based index 기준: 0, 2, 4, ..., 28 → 총 15줄)
    reduced_lines = lines[::2]

    with open(dst_path, 'w') as f:
        f.writelines(reduced_lines)

    if (i + 1) % 500 == 0 or (i + 1) == len(file_list):
        print(f"✅ 처리 완료: {i + 1}/{len(file_list)}")

print(f"\n🎯 총 {len(file_list)}개 파일이 완전히 원본과 동일한 형식으로 15프레임으로 저장되었습니다.")


총 변환 대상 파일 수: 1452개
✅ 처리 완료: 500/1452
✅ 처리 완료: 1000/1452
✅ 처리 완료: 1452/1452

🎯 총 1452개 파일이 완전히 원본과 동일한 형식으로 15프레임으로 저장되었습니다.


### 라벨별로 증강하고, 총 2000개 이상 확보하는 코드

In [17]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from collections import defaultdict

# 기본 설정
source_dir = '/home/jieun/Gesture_Recognition/test_data'  # 원본 CSV 폴더
target_dir = '/home/jieun/Gesture_Recognition/train_data_augmented'  # 증강 결과 저장 폴더
os.makedirs(target_dir, exist_ok=True)

# 증강할 타겟 라벨
target_labels = ['Turn Clockwise', 'Turn Anticlockwise', 'Stop', 'None', 'Play']  # Turn Clockwise, Turn AntiClockwise, Stop, None, Play

# 증강 함수
def augment_shift_noise(data):
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    noise = np.random.normal(0, 0.02, size=data[:, :-1].shape)  # 마지막 열은 라벨이므로 제외
    data[:, :-1] += noise
    return data

def augment_shift_scale(data):
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    scale = np.random.uniform(0.95, 1.05)
    data[:, :-1] *= scale
    return data

def augment_repeat_insert(data):
    insert_idx = random.randint(0, len(data)-1)
    frame_to_insert = data[insert_idx:insert_idx+1]
    insert_pos = random.randint(0, len(data))
    data = np.insert(data, insert_pos, frame_to_insert, axis=0)
    if len(data) > 30:
        data = data[:30]
    return data

# 원본 파일 읽기 (파일명 기준으로 필터링)
label_to_files = defaultdict(list)
for fname in os.listdir(source_dir):
    if fname.endswith('.csv'):
        lower_name = fname.lower()
        for keyword in target_labels:
            if keyword.lower() in lower_name:
                path = os.path.join(source_dir, fname)
                data = pd.read_csv(path, header=None).values
                label_to_files[keyword].append((fname, data))
                break

# 각 라벨별로 2000개가 되도록 증강
file_index = 1
# 원본 저장 및 증강
for label_name in target_labels:
    original = label_to_files[label_name]
    count = len(original)
    print(f"🔹 Label {label_name}: 원본 {count}개")

    for fname, arr in original:
        save_path = os.path.join(target_dir, f'{file_index:05d}_{label_name}.csv')
        np.savetxt(save_path, arr, delimiter=',')
        file_index += 1

    while count < 2000:
        _, base = random.choice(original)
        aug_type = random.choice(['noise', 'scale', 'repeat'])
        if aug_type == 'noise':
            aug = augment_shift_noise(base.copy())
        elif aug_type == 'scale':
            aug = augment_shift_scale(base.copy())
        else:
            aug = augment_repeat_insert(base.copy())
        save_path = os.path.join(target_dir, f'{file_index:05d}_{label_name}.csv')
        np.savetxt(save_path, aug, delimiter=',')
        file_index += 1
        count += 1

    print(f"✅ Label {label_name} → 총 {count}개 확보 완료")

print(f"\n🎉 전체 생성 파일 수: {file_index - 1}개")


🔹 Label Turn Clockwise: 원본 13개
✅ Label Turn Clockwise → 총 2000개 확보 완료
🔹 Label Turn Anticlockwise: 원본 13개
✅ Label Turn Anticlockwise → 총 2000개 확보 완료
🔹 Label Stop: 원본 13개
✅ Label Stop → 총 2000개 확보 완료
🔹 Label None: 원본 13개
✅ Label None → 총 2000개 확보 완료
🔹 Label Play: 원본 13개
✅ Label Play → 총 2000개 확보 완료

🎉 전체 생성 파일 수: 10000개


In [1]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# 경로 설정
source_dir = '/home/jieun/Gesture_Recognition/train_data_augmented'
target_dir = '/home/jieun/Gesture_Recognition/train_data_allstop_15'
os.makedirs(target_dir, exist_ok=True)

# 숫자_라벨명.csv 형태에만 매칭되는 정규표현식
filename_pattern = re.compile(r'^\d+_[a-zA-Z]+\.csv$')

# 홀수 행만 추출하고 15프레임으로 자르는 함수
def reduce_to_15_odd_rows(data):
    reduced = data[::2]  # 0, 2, 4,... (1,3,5,... 번째 행)
    return reduced[:15]

# 변환 시작
for fname in tqdm(os.listdir(source_dir), desc="15프레임 변환 중"):
    if fname.endswith('.csv') and filename_pattern.match(fname):
        path = os.path.join(source_dir, fname)
        data = pd.read_csv(path, header=None).values
        reduced = reduce_to_15_odd_rows(data)
        save_path = os.path.join(target_dir, fname)
        np.savetxt(save_path, reduced, delimiter=',')


15프레임 변환 중: 100%|██████████| 15449/15449 [00:13<00:00, 1122.92it/s]


In [2]:
import os
import numpy as np
import pandas as pd
import re

# 경로 설정
source_dir = '/home/jieun/Gesture_Recognition/train_data_augmented'
target_dir = '/home/jieun/Gesture_Recognition/train_data_allstop_15'
os.makedirs(target_dir, exist_ok=True)

# 타겟 라벨
target_labels = ['Turn Clockwise', 'Turn Anticlockwise']

# 정규표현식으로 숫자_라벨명.csv 형식 필터링
pattern = re.compile(r'^\d+_(.+)\.csv$')

# 처리 시작
for fname in os.listdir(source_dir):
    if not fname.endswith('.csv'):
        continue

    match = pattern.match(fname)
    if match:
        label = match.group(1)
        if label in target_labels:
            path = os.path.join(source_dir, fname)
            data = pd.read_csv(path, header=None).values

            # 홀수 행만 추출 (0, 2, 4, ..., 최대 15개)
            reduced = data[::2]

            save_path = os.path.join(target_dir, fname)
            np.savetxt(save_path, reduced, delimiter=',')

print("✅ 변환 완료되었습니다.")


✅ 변환 완료되었습니다.


### 학습 데이터로 사용했던 라벨만 다시 데이터 증강 (test용)
- 원본 파일은 저장하지 않음
- noise, scale, brightness 조합
- 'noise', 'scale', 'brightness', 'noise+scale', 'noise+brightness', 'scale+brightness' - 중 한가지 랜덤

In [4]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

# 경로 설정
source_dir = '/home/jieun/Gesture_Recognition/test_data'
target_dir = '/home/jieun/Gesture_Recognition/test_data_augmented_130each'
os.makedirs(target_dir, exist_ok=True)

# 증강 대상 키워드
target_keywords = ['None', 'Stop', 'Turn Clockwise', 'Turn Anticlockwise', 'Play']

# 증강 함수들
def augment_noise(data):
    noise = np.random.normal(0, 0.015, size=data[:, :-1].shape)
    data[:, :-1] += noise
    return data

def augment_scale(data):
    scale = np.random.uniform(0.97, 1.03)
    data[:, :-1] *= scale
    return data

def augment_brightness_shift(data):
    shift = np.random.uniform(-0.02, 0.02)
    data[:, 2::3] += shift  # z축만 조정
    return data

# 증강 실행 (원본은 저장하지 않음)
file_index = 1
for fname in tqdm(sorted(os.listdir(source_dir)), desc="🎯 증강 진행 중"):
    if not fname.endswith('.csv'):
        continue
    if not any(keyword.lower() in fname.lower() for keyword in target_keywords):
        continue

    fpath = os.path.join(source_dir, fname)
    base = pd.read_csv(fpath, header=None).values

    # 증강 10개 생성
    for _ in range(10):
        aug = base.copy()
        choice = random.choice(['noise', 'scale', 'brightness', 'noise+scale', 'noise+brightness', 'scale+brightness'])

        if 'noise' in choice:
            aug = augment_noise(aug)
        if 'scale' in choice:
            aug = augment_scale(aug)
        if 'brightness' in choice:
            aug = augment_brightness_shift(aug)

        save_path = os.path.join(target_dir, f'{file_index:05d}_{fname}')
        np.savetxt(save_path, aug, delimiter=',')
        file_index += 1

print(f"\n✅ 총 {file_index - 1}개 증강 파일 생성 완료!")


🎯 증강 진행 중: 100%|██████████| 143/143 [00:00<00:00, 152.49it/s]


✅ 총 650개 증강 파일 생성 완료!


### 직원 데이터 중 Left, Right, Slide, Twinkle, Rock, Okay 제스처만 다시 증강
- 각 원본 파일당 10개씩 증강
- 원본 파일은 포함하지 않음
- noise, scale, ramdon insert 중 한가지 랜덤 적용

In [7]:
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

# 경로 설정
source_dir = '/home/jieun/Gesture_Recognition/test_data'
target_dir = '/home/jieun/Gesture_Recognition/test_data_augmented_130each'
os.makedirs(target_dir, exist_ok=True)

# 제외할 키워드 (대소문자/띄어쓰기 일치 필수)
exclude_keywords = [
    'Stop',
    'None',
    'Turn Clockwise',
    'Turn Anticlockwise',
    'Play'
]

# 증강 함수 정의
def augment_shift_noise(data):
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    noise = np.random.normal(0, 0.02, size=data[:, :99].shape)
    data[:, :99] += noise
    return data

def augment_shift_scale(data):
    shift = random.randint(-2, 2)
    data = np.roll(data, shift, axis=0)
    scale = np.random.uniform(0.95, 1.05)
    data[:, :99] *= scale
    return data

def augment_repeat_insert(data):
    insert_idx = random.randint(0, len(data) - 1)
    frame_to_insert = data[insert_idx:insert_idx + 1]
    insert_pos = random.randint(0, len(data))
    data = np.insert(data, insert_pos, frame_to_insert, axis=0)
    if len(data) > 30:
        data = data[:30]
    return data

# 증강 시작
file_list = sorted([
    f for f in os.listdir(source_dir)
    if f.endswith('.csv') and not any(keyword in f for keyword in exclude_keywords)
])

file_index = 1

for fname in tqdm(file_list, desc="✅ 제스처 증강 중"):
    fpath = os.path.join(source_dir, fname)
    base = pd.read_csv(fpath, header=None).to_numpy()

    # 제스처 이름 추출 (예: "data_gesture_rock.csv" → "rock")
    gesture_name = fname.replace('.csv', '').split('_')[-1]

    # 증강 10개 (원본 저장 X)
    for i in range(10):
        aug_type = random.choice(['noise', 'scale', 'repeat'])
        if aug_type == 'noise':
            aug = augment_shift_noise(base.copy())
        elif aug_type == 'scale':
            aug = augment_shift_scale(base.copy())
        else:
            aug = augment_repeat_insert(base.copy())

        save_path = os.path.join(target_dir, f'data_gesture_{file_index:04d}_{gesture_name}.csv')
        np.savetxt(save_path, aug, delimiter=',')
        file_index += 1

print(f"\n🎉 총 생성된 증강 파일 수: {file_index - 1}개")

✅ 제스처 증강 중: 100%|██████████| 78/78 [00:01<00:00, 69.03it/s]


🎉 총 생성된 증강 파일 수: 780개
